In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit

/Users/vega7unk/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
RAW_PATH      = "../data/raw/MobiAct_combined.csv"
OUT_FOLDER    = "samples"
OUT_FILENAME  = "sample_fall_window.csv"
WINDOW_SIZE   = 500
STEP_SIZE     = 250


In [3]:
dtype_map = {
    "subject_id": "int16",
    "trial":      "int16",
    "acc_x":      "float32", "acc_y":      "float32", "acc_z":      "float32",
    "gyro_x":     "float32", "gyro_y":     "float32", "gyro_z":     "float32",
    "azimuth":    "float32", "pitch":      "float32", "roll":       "float32",
    "label":      "category"
}
sensor_cols = [
    "acc_x","acc_y","acc_z",
    "gyro_x","gyro_y","gyro_z",
    "azimuth","pitch","roll"
]

In [4]:
df = pd.read_csv(RAW_PATH, dtype=dtype_map, engine="c")

In [5]:
groups = df["subject_id"].values
gss1   = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
trainval_idx, test_idx = next(gss1.split(df, df["label"], groups=groups))

In [6]:
test_subjects = df.iloc[test_idx]["subject_id"].unique()
df_test       = df[df["subject_id"].isin(test_subjects)].reset_index(drop=True)

In [7]:
fall_set     = {"BSC","FKL","SDL","FOL"}
post_fall    = {"LYI"}

In [8]:
def map_label(x):
    if x in fall_set:
        return "FALL"
    elif x in post_fall:
        return "POST_FALL"
    else:
        return "ADL"

df_test["fall_label"] = df_test["label"].apply(map_label)


In [9]:
def create_windows(X_df, y_ser, window_size, step_size):
    Xw, yw = [], []
    data   = X_df.values
    labels = y_ser.values
    n      = len(X_df)
    for start in range(0, n - window_size + 1, step_size):
        block       = data[start : start + window_size]
        block_lbls  = labels[start : start + window_size]
        uniq        = set(block_lbls)
        if "FALL" in uniq:
            lbl = "FALL"
        elif "ADL" in uniq:
            lbl = "ADL"
        else:
            continue
        Xw.append(block)
        yw.append(lbl)
    return np.array(Xw), np.array(yw)

In [10]:
X_raw, y_raw = create_windows(
    df_test[sensor_cols],
    df_test["fall_label"],
    WINDOW_SIZE, STEP_SIZE
)

In [11]:
fall_indices = np.where(y_raw == "FALL")[0]
if len(fall_indices) == 0:
    raise RuntimeError("No FALL windows found in the raw test set!")

first_fall = X_raw[fall_indices[0]]  # shape: (WINDOW_SIZE, n_features)
df_sample  = pd.DataFrame(first_fall, columns=sensor_cols)


In [12]:

os.makedirs(OUT_FOLDER, exist_ok=True)
out_path = os.path.join(OUT_FOLDER, OUT_FILENAME)
df_sample.to_csv(out_path, index=False)

print(f"Wrote raw FALL window to: {out_path} ({df_sample.shape[0]} rows × {df_sample.shape[1]} cols)")

Wrote raw FALL window to: samples/sample_fall_window.csv (500 rows × 9 cols)
